In [ ]:
import cv2
import dlib
import os
import time
import numpy as np

In [ ]:
# Load the detector
detector = dlib.get_frontal_face_detector()

# Load the predictor
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

path = "img"
files = []

for r, d, f in os.walk(path):
    for file in f:
        if '.jpg' in file:
            files.append(os.path.join(r, file))

In [ ]:
def facial_analysis(path):
    img = cv2.imread(path)

    # Convert image into grayscale
    gray = cv2.cvtColor(src=img, code=cv2.COLOR_BGR2GRAY)

    #dim = (int(img.shape[1] / 4), int(img.shape[0] / 4))
    #grey = cv2.resize(gray, dim, interpolation = cv2.INTER_AREA)
    
    # Use detector to find landmarks
    faces = detector(gray)

    landmarks = None
    
    if (len(faces) > 0):
        face = faces[0]
        x1 = face.left() # left point
        y1 = face.top() # top point
        x2 = face.right() # right point
        y2 = face.bottom() # bottom point
        # Create landmark object
        landmarks = predictor(image=gray, box=face)
        # Loop through all the points
        '''
        for n in range(0, 68):
            x = landmarks.part(n).x
            y = landmarks.part(n).y
            # Draw a circle
            cv2.circle(img=img, center=(x, y), radius=3, color=(0, 255, 0), thickness=-1)
        '''
        # show the image
        #cv2.imshow(winname="Face", mat=grey)
        
        # Delay between every fram
        #cv2.waitKey(delay=0)
        
        # Close all windows
        #cv2.destroyAllWindows()

    return landmarks

In [ ]:
imageCount = len(files)

allLandmarks = []
allLeftEyes = np.zeros((imageCount, 2))
allRightEyes = np.zeros((imageCount, 2))
allChins = np.zeros((imageCount, 2))

start = time.time()
for i in range(imageCount):
    print(f"file: {i} of {len(files)}", end='\r')
    landmarks = facial_analysis(files[i])
    if (landmarks != None):
        allLandmarks.append(landmarks)
        
        allLeftEyes[i, 0] = allLandmarks[-1].part(45).x
        allLeftEyes[i, 1] = allLandmarks[-1].part(45).y
    
        allRightEyes[i, 0] = allLandmarks[-1].part(36).x
        allRightEyes[i, 1] = allLandmarks[-1].part(36).y
    
        allChins[i, 0] = allLandmarks[-1].part(8).x
        allChins[i, 1] = allLandmarks[-1].part(8).y
end = time.time()
print(end - start)

In [ ]:
allLeftEyes[allLeftEyes == 0] = np.nan
allRightEyes[allRightEyes == 0] = np.nan
allChins[allChins == 0] = np.nan


leftEyesMean = np.nanmean(allLeftEyes, axis=0)
rightEyesMean = np.nanmean(allRightEyes, axis=0)
chinMean = np.nanmean(allChins, axis=0)

print(leftEyesMean, rightEyesMean, chinMean)

In [ ]:
srcTri = np.zeros((3, 2))
dstTri = np.zeros((3, 2))
dim = (0, 0)

downScale = 4

dstTri[0] = leftEyesMean
dstTri[1] = rightEyesMean
dstTri[2] = chinMean

dstTri = (dstTri / downScale).astype(np.float32)

transImgs = []

for i in range(len(allLeftEyes)):
    srcTri[0] = allLeftEyes[i]
    srcTri[1] = allRightEyes[i]
    srcTri[2] = allChins[i]

    warpMat = cv2.getAffineTransform((srcTri / downScale).astype(np.float32), dstTri)

    img = cv2.imread(files[i])

    dim = (int(img.shape[1] / downScale), int(img.shape[0] / downScale))
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)

    img = cv2.warpAffine(img, warpMat, (img.shape[1], img.shape[0]))
    
    transImgs.append(img)

In [ ]:
out = cv2.VideoWriter('project.avi',cv2.VideoWriter_fourcc(*'DIVX'), 30, dim)

for i in range(len(transImgs)):
    out.write(transImgs[i])
out.release()